# 4.1模型构造

Module 类是 nn 模块⾥提供的⼀个模型构造类，是所有神经⽹络模块的基类，我们可以继承它来定义我们想要的模型。下⾯继承 Module 类构造本节开头提到的多层感知机。这⾥定义的 MLP 类᯿载了Module 类的 __init__ 函数和 forward 函数。它们分别⽤于创建模型参数和定义前向计算。前向计
算也即正向传播。

In [5]:
import torch
from torch import nn

class MLP(nn.Module):
    # 声明带有模型参数的层  这里声明了两个全连接层
    def __init__(self,**kwargs):
        # 调用MLP父类Block的构造函数来进行必要的初始化
        # 这样在构造实例时还可以指定其他函数
        super(MLP,self).__init__(**kwargs)
        self.hidden = nn.Linear(784,256)  #隐藏层
        self.act = nn.ReLU()
        self.output = nn.Linear(256,10)  #输出层
        
        # 定义前向计算  
    def forward(self,x):
        a = self.act(self.hidden(x))
        return self.output(a)
        

In [7]:
X = torch.rand(2,784)
net = MLP()
print(net)
b = net(X)
b.shape

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


torch.Size([2, 10])

## module 的子类
### Sequential类
它可以接收⼀个⼦模块的有序字典（OrderedDict）或者⼀系列⼦模块作
为参数来逐⼀添加 Module 的实例，⽽模型的前向计算就是将这些实例按添加的顺序逐⼀计算。
下⾯我们实现⼀个与 Sequential 类有相同功能的 MySequential 类。

In [8]:
class MySequential(nn.Module):
    from collections import OrderedDict
    def __init__(self,*args):
        super(MySequential,self).__init__()
        if len(args) == 1 and isinstance(args[0],OrderedDict):
            #如果传入的是一个OrderedDict
            for key,module in args[0].items():
                self.add_module(key,module)
                # add_module⽅法会将module添加进self._modules(⼀个OrderedDict)
        else: #如果传入的是一些Module
            for idx,module in enumerate(args):
                self.add_module(str(idx),module)
    def forward(self,input):
        #self._module返回一个OrderedDict 
        # 保证会按照成员添加是的顺序遍历
        for module in self._modules.values():
            input = module(input)
        return input

Sequential ModuleList ModuleDict 都可以方便的添加神经网络层 且不需要添加forward函数

## 4.1.3 构造复杂的模型

虽然上⾯介绍的这些类可以使模型构造更加简单，且不需要定义 forward 函数，但直接继承 Module类可以极⼤地拓展模型构造的灵活性。
下⾯我们构造⼀个稍微复杂点的⽹络 FancyMLP 。在这个⽹络中，我们通过 get_constant 函数创建训练中不被迭代的参数，即常数参数。
在前向计算中，除了使⽤创建的常数参数外，我们还使⽤ Tensor 的函数和Python的控制流，并多次调⽤相同的层。

In [15]:
class FancyMLP(nn.Module):
    def __init__(self,**kwargs):
        super(FancyMLP,self).__init__(**kwargs)
        
        self.rand_weight = torch.rand((20,20),requires_grad=False)  
        # 不可训练参数 常数参数
        self.linear = nn.Linear(20, 20)
    def forward(self,x):
        x = self.linear(x)
        # 使用创建的常数参数 以及nn.functional中的relu函数和mm函数
        x = nn.functional.relu(torch.mm(x,self.rand_weight.data) +1)
        # 复⽤全连接层。等价于两个全连接层共享参数
        x = self.linear(x)
        # 控制流 这里我们需要调用item函数来返回标量进行比较
        while x.norm().item() >1:
#             print(x.norm().item())
            x/=2
        if x.norm().item()<0.8:
            x*=10
        return x.sum()

In [22]:
X = torch.rand(2, 20)
net = FancyMLP()
print(net)
net(X)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(-6.5763, grad_fn=<SumBackward0>)

因为 FancyMLP 和 Sequential 类都是 Module 类的⼦类，所以我们可以嵌套调⽤它们。

In [24]:
class NestMLP(nn.Module):
    def __init__(self,**kwargs):
        super(NestMLP,self).__init__(**kwargs)
        self.net = nn.Sequential(nn.Linear(40,30),nn.ReLU())
    def forward(self,x):
        return self.net(x)
net = nn.Sequential(NestMLP(),nn.Linear(30,20),FancyMLP())
X= torch.rand(2,40)
print(net)

Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): FancyMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


In [25]:
net(X)

tensor(9.8350, grad_fn=<SumBackward0>)

# 4.2 模型参数的访问 初始化和共享

In [39]:
import torch
from torch import nn
from torch.nn import init
net = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3, 1)) #
# pytorch已进⾏默认初始化
print(net) 
X = torch.rand(2, 4) 
Y = net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


## 4.2.1访问模型参数

In [40]:
print(type(net.named_parameters())) 
#以迭代器的形式返回

<class 'generator'>


In [41]:
for name, param in net.named_parameters():
     print(name, param.size())

0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [42]:
for name, param in net[0].named_parameters():
     print(name, param.size())

weight torch.Size([3, 4])
bias torch.Size([3])


param 的 类 型为 torch.nn.parameter.Parameter 
其实这是 Tensor 的⼦类，
和 Tensor 不同的是如果⼀个 Tensor 是 Parameter 
那么它会⾃动被添加到模型的参数列表⾥

In [43]:
class MyModel(nn.Module):
    def __init__(self,**kwargs):
        super(MyModel,self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20,20))
        self.weight2 = torch.rand(20,20)
    def forward(self,x):
        pass
n = MyModel()
for name,param in n.named_parameters():
    print(name)

weight1


In [46]:
weight_0 = list(net[0].parameters())[0]
print(weight_0)
print(weight_0.data)
print(weight_0.grad) # 反向传播前梯度为None
Y.backward(retain_graph=True)
print(weight_0.grad)

#  出现下述错误是因为 在经过一次计算后，梯度已经被释放
# 因此再次计算机会找不到梯度 
# 解决方法是在backward(retain_graph = True)

Parameter containing:
tensor([[ 0.4576,  0.3284, -0.4134, -0.1837],
        [-0.2020,  0.0932,  0.2688,  0.3181],
        [ 0.2389, -0.3934, -0.3446,  0.2894]], requires_grad=True)
tensor([[ 0.4576,  0.3284, -0.4134, -0.1837],
        [-0.2020,  0.0932,  0.2688,  0.3181],
        [ 0.2389, -0.3934, -0.3446,  0.2894]])
tensor([[0.6855, 1.3193, 1.1676, 0.1888],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000]])
tensor([[1.0283, 1.9789, 1.7513, 0.2831],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000]])


## 4.2.2 初始化模型参数

In [48]:
for name, param in net.named_parameters():
     if 'weight' in name:
         init.normal_(param, mean=0, std=0.01)
         print(name, param.data)
#          print(param.data.sum())

0.weight tensor([[-0.0235,  0.0128, -0.0161,  0.0033],
        [ 0.0134,  0.0017, -0.0179, -0.0047],
        [ 0.0004,  0.0118,  0.0100, -0.0092]])
tensor(-0.0180)
2.weight tensor([[-0.0174,  0.0182, -0.0186]])
tensor(-0.0179)


In [50]:
# 使用常数来初始化权重参数
for name,param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param,val = 1)
        print(name,param.data)

0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


## 4.2.3 自定义初始化方法

In [51]:
def normal_(temsor,mean=0,std=1):
    with torch.no_grad():
        return tensor.normal_(mean,std)

def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10.10)
        tensor *=(tensor.abs()>5).float()
        
for name,param in net.named_parameters():
     if 'weight' in name:
         init_weight_(param)
         print(name, param.data)
#          print(param.data.sum())

0.weight tensor([[ 0.0000,  0.0000, -0.0000, -7.1228],
        [-0.0000, -6.3637, -0.0000, -0.0000],
        [-6.1078,  0.0000, -9.7701, -7.5145]])
2.weight tensor([[-0., -0., 0.]])


In [58]:
with torch.no_grad():
    param.uniform_(-10,10)
    c=(param.abs()>5).float()
param.data

tensor([6.1493])

## 4.2.4 共享模型参数
如果我们传⼊ Sequential 的模块是同⼀个 Module 实例
参数也是共享的，

In [59]:
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear)
print(net)
for name, param in net.named_parameters():
     init.constant_(param, val=3)
     print(name, param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [61]:
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))
# 在内存中 这两个线性层其实是一个对象

True
True


In [62]:
# 因为模型参数⾥包含了梯度，
# 所以在反向传播计算时，这些共享的参数的梯度是累加的:
x = torch.ones(1, 1) 
y = net(x).sum()
print(y) 
y.backward()
print(net[0].weight.grad)# 单次梯度是3，两次所以就是6

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])


# 4.3 模型参数的延后初始化
由于使⽤Gluon创建的全连接层的时候不需要指定输⼊个数。所以当调⽤ initialize 函数时，由于隐藏层输⼊个数依然未知，系统也⽆法得知该层权参数的形状。只有在当形状已知的输⼊ X 传进⽹络做前向计算 net(X) 时，系统才推断出该层的权重参数形状为多少，此时才进⾏真正的初始化操作。但是使⽤PyTorch在定义模型的时候就要指定输⼊的形状，所以也就不存在这个问题了

# 4.4 自定义层
## 4.4.1 不含模型参数的自定义层

In [64]:
# 下⾯的 CenteredLayer 类通过继承 Module 类
# ⾃定义了⼀个将输⼊减掉均值后输出的层
# 并将层的计算定义在了 forward 函数⾥
# 这个层⾥不含模型参数。

import torch 
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self,**kwargs):
        super(CenteredLayer,self).__init__(**kwargs)
    def forward(self,x):
        return x - x.mean()

In [67]:
layer = CenteredLayer()
layer(torch.tensor([1,2,3,4,5],dtype = torch.float))

tensor([-2., -1.,  0.,  1.,  2.])

In [71]:
net = nn.Sequential(nn.Linear(8,128),CenteredLayer())

In [74]:
y = net(torch.rand(4,8))
y.mean().item()

-1.862645149230957e-09

## 含模型参数的自定义层
了 Parameter 类其实是 Tensor 的⼦类，如果⼀个 Tensor 是 Parameter ，那么它会⾃动被添加到模型的参数列表⾥。
所以在⾃定义含模型参数的层时，我们应该将参数定义成 Parameter 
除了像4.2.1节那样直接定义成 Parameter 类外
还可以使⽤ ParameterList 和 ParameterDict 分别定义参数的列表和字典。

In [77]:
# ParameterList 接收⼀个 Parameter 实例的列表作为输⼊然后得到⼀个参数列表
# 使⽤的时候可以⽤索引来访问某个参数
# 另外也可以使⽤ append 和 extend 在列表后⾯新增参数。

class MyDense(nn.Module):
    def __init__(self):
        super(MyDense,self).__init__()
        self.params = nn.ParameterList([nn.Parameter(torch.randn(4,4)) for i in range(3)])
        self.params.append(nn.Parameter(torch.randn(4,1)))
    
    def forward(self,x):
        for i in range(len(self.params)):
            x = torch.mm(x,self.params[i])
        return x
net = MyDense()
print(net)

MyDense(
  (params): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 4x4]
      (1): Parameter containing: [torch.FloatTensor of size 4x4]
      (2): Parameter containing: [torch.FloatTensor of size 4x4]
      (3): Parameter containing: [torch.FloatTensor of size 4x1]
  )
)


In [79]:
class MyDictDense(nn.Module):
    def __init__(self):
        super(MyDictDense,self).__init__()
        self.params = nn.ParameterDict({
            'linear1':nn.Parameter(torch.randn(4,4)),
            'linear2':nn.Parameter(torch.randn(4,4))
        })
        self.params.update({'linear3': nn.Parameter(torch.randn(4,2))})
    def forward(self,x,choice = 'linear1'):
        return torch.mm(x,self.params[choice])

net = MyDictDense()
print(net)

MyDictDense(
  (params): ParameterDict(
      (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
      (linear2): Parameter containing: [torch.FloatTensor of size 4x4]
      (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
  )
)


In [81]:
x = torch.ones(1,4)
print(net(x,'linear1'))
print(net(x,'linear2'))
print(net(x,'linear3'))

tensor([[-1.1341,  0.0853, -2.2677, -0.0330]], grad_fn=<MmBackward>)
tensor([[2.1894, 0.9286, 0.9847, 3.4789]], grad_fn=<MmBackward>)
tensor([[-0.4577,  0.6231]], grad_fn=<MmBackward>)


In [83]:
net = nn.Sequential(
     MyDictDense(),
     MyDense(),
)
print(net)

Sequential(
  (0): MyDictDense(
    (params): ParameterDict(
        (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
        (linear2): Parameter containing: [torch.FloatTensor of size 4x4]
        (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
    )
  )
  (1): MyDense(
    (params): ParameterList(
        (0): Parameter containing: [torch.FloatTensor of size 4x4]
        (1): Parameter containing: [torch.FloatTensor of size 4x4]
        (2): Parameter containing: [torch.FloatTensor of size 4x4]
        (3): Parameter containing: [torch.FloatTensor of size 4x1]
    )
  )
)


In [84]:
print(net(x))

tensor([[-145.4057]], grad_fn=<MmBackward>)


# 4.5 读取和存储
我们把内存中训练好的模型参数存储在硬盘上供后续读取使用
## 4.5.1 读写tensor
我们可以直接使⽤ save 函数和 load 函数分别存储和读取 Tensor 。 save 使⽤Python的pickle实⽤
程序将对象进⾏序列化，然后将序列化的对象保存到disk，使⽤ save 可以保存各种对象,包括模型、张
量和字典等。⽽ laod 使⽤pickle unpickle⼯具将pickle的对象⽂件反序列化为内存。

In [85]:
import torch
from torch import nn

x = torch.ones(3)
torch.save(x,'x.pt')

In [86]:
x2 = torch.load('x.pt')
x2

tensor([1., 1., 1.])

In [87]:
y= torch.zeros(4)
torch.save([x,y],'xy.pt')
xy_list = torch.load('xy.pt')
xy_list

[tensor([1., 1., 1.]), tensor([0., 0., 0., 0.])]

In [88]:
torch.save({'x':x,'y':y},'xy_dict.pt')
xy = torch.load('xy_dict.pt')
xy

{'x': tensor([1., 1., 1.]), 'y': tensor([0., 0., 0., 0.])}

## 4.5.2 读写模型
state_dict 是⼀个从参数名称隐射到参数 Tesnor 的字典对象。

In [89]:
class MLP(nn.Module):
     def __init__(self):
         super(MLP, self).__init__()
         self.hidden = nn.Linear(3, 2)
         self.act = nn.ReLU()
         self.output = nn.Linear(2, 1)
     def forward(self, x):
         a = self.act(self.hidden(x))
         return self.output(a)

In [91]:
NET  = MLP()
NET.state_dict()
#注意，只有具有可学习参数的层(卷积层、线性层等)
# 才有 state_dict 中的条⽬。
# 优化器( optim )也有⼀个 state_dict 
# 其中包含关于优化器状态以及所使⽤的超参数的信息。

OrderedDict([('hidden.weight',
              tensor([[-0.3664,  0.4260,  0.1871],
                      [ 0.2061, -0.3556,  0.3939]])),
             ('hidden.bias', tensor([-0.0040,  0.0298])),
             ('output.weight', tensor([[0.4334, 0.6145]])),
             ('output.bias', tensor([-0.2200]))])

In [93]:
optimizer = torch.optim.SGD(NET.parameters(),lr = 0.001,momentum = 0.9)
optimizer.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.001,
   'momentum': 0.9,
   'dampening': 0,
   'weight_decay': 0,
   'nesterov': False,
   'params': [0, 1, 2, 3]}]}

### 4.5.2.2 保存和加载模型
#### 推荐保存和加载state_dict
torch.save(model.state_dict(),PATH)

In [95]:
X = torch.rand(2,3)
Y = NET(X)

Path = "./net.pth"
torch.save(NET.state_dict(),Path)

net2 = MLP()
net2.load_state_dict(torch.load(Path))
Y2 = net2(X)
Y2 == Y

tensor([[True],
        [True]])

# 4.6 GPU计算
到⽬前为⽌，我们⼀直在使⽤CPU计算。
对复杂的神经⽹络和⼤规模的数据来说，使⽤CPU来计算可能不够⾼效。
在本节中，我们将介绍如何使⽤单块NVIDIA GPU来计算。
所以需要确保已经安装好了PyTorch GPU版本。


In [2]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
import torch
from torch import nn
torch.cuda.is_available() # 输出 True

True

In [4]:
torch.cuda.device_count() # 输出 1

1

In [5]:
torch.cuda.current_device() # 输出 0

0

In [6]:
torch.cuda.get_device_name(0) # 输出 'GeForce GTX 1050'

'GeForce GTX 1080'

In [13]:
x = torch.tensor([1,2,3])
x = x.cuda()
x,x.device

(tensor([1, 2, 3], device='cuda:0'), device(type='cuda', index=0))

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

x = torch.tensor([1,2,3],device = device)

y = torch.tensor([3,4,5]).to(device)
x,y

(tensor([1, 2, 3], device='cuda:0'), tensor([3, 4, 5], device='cuda:0'))

In [15]:
net = nn.Linear(3,1)
list(net.parameters())[0].device

device(type='cpu')

In [16]:
net.cuda()
list(net.parameters())[0].device

device(type='cuda', index=0)

In [17]:
x= torch.rand(2,3).cuda()
net(x)

tensor([[-0.4807],
        [-0.3510]], device='cuda:0', grad_fn=<AddmmBackward>)